In [1]:
import tensorflow as tf
import numpy as np

np.set_printoptions(precision=4)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

print(tf.__version__)

2.4.0


In [2]:
pip install netpbmfile

In [3]:
!tar xvf faces.tar  >> /dev/null

In [4]:
import netpbmfile
import pathlib
from sklearn import preprocessing

data_dir = pathlib.Path("faces")
images_open = list(data_dir.glob('*/*_open_2.pgm'))
images_sunglasses = list(data_dir.glob('*/*_sunglasses_2.pgm'))

images_str = np.concatenate((images_open, images_sunglasses), axis=0)

In [5]:
y_open = np.full((len(images_open), 1), 'open')
y_sunglasses = np.full((len(images_sunglasses), 1), 'sunglasses')

y = np.concatenate((y_open, y_sunglasses), axis=0)

CLASS_NAMES = ['open', 'sunglasses']
encoder = preprocessing.LabelEncoder()
encoder.fit(CLASS_NAMES)
labels = encoder.transform(np.array(y))

X = []
for i in range(len(images_str)):
    image_arr = netpbmfile.imread(str(images_str[i]))
    # Add channels_last
    image_arr = np.expand_dims(image_arr, axis=2)
    image_arr = image_arr.astype(np.float32)
    X.append(image_arr)
    
features = np.asarray(X)

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [6]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(features, labels, test_size = 0.1, random_state = 0)

BATCH_SIZE = 32
REPEAT_SIZE = 100

def train_input_fn():
    return tf.data.Dataset.from_tensor_slices((xTrain, yTrain)).repeat(REPEAT_SIZE).batch(BATCH_SIZE)

def test_input_fn():
    return tf.data.Dataset.from_tensor_slices((xTest, yTest)).repeat(REPEAT_SIZE).batch(BATCH_SIZE)

In [7]:
from tensorflow.keras import layers

import tensorflow.compat.v1 as tf1
tf1.disable_v2_behavior()

def cnn_model_fn(features, labels, mode):    
    """Model function for CNN"""    
    
    layer = layers.Conv2D(32, 3, padding='same', activation='relu')(features)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Conv2D(32, 3, padding='same', activation='relu')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.MaxPooling2D(pool_size=(2,2))(layer)
    layer = layers.Dropout(0.25)(layer)

    layer = layers.Conv2D(64, 3, padding='same', activation='relu')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Conv2D(64, 3, padding='same', activation='relu')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.MaxPooling2D(pool_size=(2,2))(layer)
    layer = layers.Dropout(0.25)(layer)

    layer = layers.Conv2D(128, 3, padding='same', activation='relu')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Conv2D(128, 3, padding='same', activation='relu')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Conv2D(128, 3, padding='same', activation='relu')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.MaxPooling2D(pool_size=(2,2))(layer)
    layer = layers.Dropout(0.25)(layer)
    
    #Fully connected layer    
    flatten = layers.Flatten()(layer)                    
    dense1 = layers.Dense(flatten.get_shape()[1], activation='relu')(flatten)
    batch2 = layers.BatchNormalization()(dense1)
    drop2 = layers.Dropout(0.25)(batch2)    
    
    
    # Logits Layer
    logits = tf.keras.layers.Dense(units=len(CLASS_NAMES))(drop2)  
    
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate Loss   
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)    
    loss = tf.reduce_mean(loss)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf1.train.GradientDescentOptimizer(learning_rate=0.0005)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf1.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metrics Evaluation mode
    eval_metric_ops = {
        "accuracy": tf1.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
# Create the Estimator
import tempfile
model_dir = tempfile.mkdtemp()
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir=model_dir)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpkp4i55ba', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.estimator.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

In [10]:
classifier.train(
    input_fn=train_input_fn,
    steps=5000,
    hooks=[logging_hook])

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpkp4i55ba/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:probabilities = [[0.8861 0.1139]
 [0.8522 0.1478]
 [0.8528 0.1472]
 [0.8711 0.1289]
 [0.8726 0.1274]
 [0.8912 0.1088]
 [0.8678 0.1322]
 [0.8945 0.1055]
 [0.8598 0.1402]
 [0.8241 0.1759]
 [0.8715 0.1285]
 [0.8824 0.1176]
 [0.8176 0.1824]
 [0.843  0.157 ]
 [0.8862 0.1138]
 [0.8741 0.1259]
 [0.8262 0.1738]
 [0.8484 0.1516]
 [0.892  0.108 ]
 [0.9219 0.0781]
 [0.836  0.164 ]

In [11]:
eval_results = classifier.evaluate(input_fn=test_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-02-11T04:05:58Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpkp4i55ba/model.ckpt-1754
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 93.43639s
INFO:tensorflow:Finished evaluation at 2021-02-11-04:07:32
INFO:tensorflow:Saving dict for global step 1754: accuracy = 0.95238096, global_step = 1754, loss = 0.07515864
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1754: /tmp/tmpkp4i55ba/model.ckpt-1754
{'accuracy': 0.95238096, 'loss': 0.07515864, 'global_step': 1754}
